In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import gmaps
import requests
import json

# Google developer API key
from appid import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
# Import the previously saved dataframe from WeatherPy
vacation_df = pd.read_csv('../WeatherPy/Data/Weather_Data.csv')

# Display the dataframe with 672 unique cities
vacation_df

,Unnamed: 0,City ID,City Name,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,3382160.0,cayenne,4.9333,-52.3333,77.59,94.0,90.0,5.75
1,2,5106841.0,albany,42.6001,-73.9662,55.89,89.0,88.0,2.55
2,3,4030556.0,rikitea,-23.1203,-134.9692,75.83,69.0,74.0,9.80
3,4,2514651.0,los llanos de aridane,28.6585,-17.9182,64.40,63.0,0.0,3.44
4,5,202061.0,kigali,-1.9500,30.0588,64.18,94.0,40.0,5.75
...,...,...,...,...,...,...,...,...,...
667,1484,3393536.0,olinda,-8.0089,-34.8553,75.20,88.0,40.0,5.75
668,1489,2022129.0,kichera,55.9375,110.0997,33.64,45.0,20.0,2.19
669,1490,1586443.0,ca mau,9.1769,105.1500,93.27,48.0,98.0,10.40
670,1493,3354077.0,opuwo,-18.0607,13.8400,63.54,60.0,34.0,3.11


In [3]:
# Creating the heat map

# 1. Storing latitudes and longitudes
locations = vacation_df[['Latitude', 'Longitude']]

# Defining variable to be used in creating heatmap
humidity = vacation_df['Humidity (%)'].astype(float)

# Plot heat map
fig = gmaps.figure(center=(10, 5), zoom_level=2, map_type='HYBRID')

# Create heat layer, with max_intensity = highest humidity = 100
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1.0)

# Add layer
fig.add_layer(heat_layer)

# I saved the heatmap by clicking on the download arrow above the map and then savinf the file in PNG format

# Display heatmap
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrowing down the dataframe

# 1. temperature range: 70 <= temp <= 80
# 2. wind speed < 10
# 3. cloudiness = 0
vacation_ideal = vacation_df[(vacation_df['Temperature (F)'] <= 80) & (vacation_df['Temperature (F)'] >= 70) &
                           (vacation_df['Wind Speed (mph)'] < 10) & 
                          (vacation_df['Cloudiness (%)']==0)]

# Display narrow-down dataframe
vacation_ideal


,Unnamed: 0,City ID,City Name,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
12,13,1067565.0,beloha,-25.1667,45.0500,76.86,58.0,0.0,5.50
30,35,2063056.0,port augusta,-32.5000,137.7667,76.80,31.0,0.0,8.05
249,397,1529660.0,aksu,41.1231,80.2644,78.80,9.0,0.0,1.36
281,461,98182.0,baghdad,33.3406,44.4009,75.20,35.0,0.0,6.91
300,504,1803365.0,linqiong,30.4159,103.4609,75.11,57.0,0.0,2.24
339,599,1280037.0,shache,38.4167,77.2406,79.12,8.0,0.0,1.63
343,606,362973.0,abnub,27.2696,31.1511,79.20,10.0,0.0,3.44
375,677,107304.0,buraydah,26.3260,43.9750,73.40,53.0,0.0,4.61


In [5]:
# Reindex the dataframe
vacation_ideal = vacation_ideal.reset_index(drop=True)
vacation_ideal

# Drop the unwanted columns
vacation_hotel = vacation_ideal.drop(columns=['Unnamed: 0', 'City ID', 'Temperature (F)',
                                              'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)'])
vacation_hotel

,City Name,Latitude,Longitude
0,beloha,-25.1667,45.0500
1,port augusta,-32.5000,137.7667
2,aksu,41.1231,80.2644
3,baghdad,33.3406,44.4009
4,linqiong,30.4159,103.4609
5,shache,38.4167,77.2406
6,abnub,27.2696,31.1511
7,buraydah,26.3260,43.9750


In [6]:
# define and create new variables to collect data on
vacation_hotel['Hotel Name'] = ""
vacation_hotel['Address'] = ""

# Display table with two new emply columns
vacation_hotel

,City Name,Latitude,Longitude,Hotel Name,Address
0,beloha,-25.1667,45.0500,,
1,port augusta,-32.5000,137.7667,,
2,aksu,41.1231,80.2644,,
3,baghdad,33.3406,44.4009,,
4,linqiong,30.4159,103.4609,,
5,shache,38.4167,77.2406,,
6,abnub,27.2696,31.1511,,
7,buraydah,26.3260,43.9750,,


In [7]:
# Finding the nearest hotels

# Set geocoordinates
target_type = 'lodging'
target_radius = 5000

# Set the parameters for the search    
params = {
    'types': target_type,
    'radius': target_radius,
    'key': gkey
}

# Using iterrows to iterate through the dataframe
for index, row in vacation_ideal.iterrows():
    
    # get latitude and longitude from the dataframe
    lat = row['Latitude']
    lng = row['Longitude']
    
    # Change the location after each iteration
    params['location'] = f"{lat},{lng}"
    
    # Base url for the search
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    # make request
    hotels = requests.get(url, params=params)
    
    # json-ify
    hotel_search = hotels.json()
    
    # use 'try . . . except' search to skip any cities with no hotels
    try:
        vacation_hotel.loc[index, 'Hotel Name'] = hotel_search['results'][0]['name']
        vacation_hotel.loc[index, 'Address'] = hotel_search['results'][0]['vicinity']
        #vacation_hotel[index, 'Country'] = hotel_search['results'][0]['country']
        
    except (KeyError, IndexError):
        print('Missing information . . . skipping')

Missing information . . . skipping


In [8]:
vacation_hotel.to_csv('vacation hotels.csv')

# Add country name
countries = ["Madagascar","Australia","China","Iraq","China","China","Egypt","Saudi Arabia"]
vacation_hotel["Country"] = countries

# Display complete table
vacation_hotel

,City Name,Latitude,Longitude,Hotel Name,Address,Country
0,beloha,-25.1667,45.0500,,,Madagascar
1,port augusta,-32.5000,137.7667,Majestic Oasis Apartments,"Marryatt Street, Port Augusta",Australia
2,aksu,41.1231,80.2644,Pudong Holiday Hotel,"交通西路1号, Jiao Tong Lu, Akesu Shi, Akesu Diqu",China
3,baghdad,33.3406,44.4009,Baghdad Hotel,Baghdad,Iraq
4,linqiong,30.4159,103.4609,Tianlun Tanxianglou Hotel,"318国道附近, 318国道, Qionglai Shi, Chengdu Shi",China
5,shache,38.4167,77.2406,Delong Hotel,"2 齐乃, Shache Xian, Ba Ge, Kashi Diqu",China
6,abnub,27.2696,31.1511,Romany Adly Gad,أمام مدرسة الثانوية الصناعية، أبنوب، مركز أبنو...,Egypt
7,buraydah,26.3260,43.9750,فندق راحة السلام,"King Abdulaziz Road, Buraydah",Saudi Arabia


In [9]:
# Plot the hotels on top of the humidity heatmap 

# Create coordinates
coordinates = vacation_hotel[['Latitude','Longitude']]

# convert the dataframe into a dictionary
vacation_hotel_dict = vacation_hotel.to_dict('records')

# Set up a template to target the information I want displayed when someone clicks on a marker
hotel_info_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Ues a list comprehension to get the information I want displayed
hotel_info = [hotel_info_template.format(**hotel) for hotel in vacation_hotel_dict]

# Add markers
markers = gmaps.marker_layer(coordinates, info_box_content=hotel_info)

# Add the markers as a layer on the heatmap
fig.add_layer(markers)

# Display the heatmap with the markers
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Just visualising the dictionry to properly identify information of interest
vacation_hotel_dict

[{'City Name': 'beloha',
  'Latitude': -25.1667,
  'Longitude': 45.05,
  'Hotel Name': '',
  'Address': '',
  'Country': 'Madagascar'},
 {'City Name': 'port augusta',
  'Latitude': -32.5,
  'Longitude': 137.7667,
  'Hotel Name': 'Majestic Oasis Apartments',
  'Address': 'Marryatt Street, Port Augusta',
  'Country': 'Australia'},
 {'City Name': 'aksu',
  'Latitude': 41.1231,
  'Longitude': 80.2644,
  'Hotel Name': 'Pudong Holiday Hotel',
  'Address': '交通西路1号, Jiao Tong Lu, Akesu Shi, Akesu Diqu',
  'Country': 'China'},
 {'City Name': 'baghdad',
  'Latitude': 33.3406,
  'Longitude': 44.4009,
  'Hotel Name': 'Baghdad Hotel',
  'Address': 'Baghdad',
  'Country': 'Iraq'},
 {'City Name': 'linqiong',
  'Latitude': 30.4159,
  'Longitude': 103.4609,
  'Hotel Name': 'Tianlun Tanxianglou Hotel',
  'Address': '318国道附近, 318国道, Qionglai Shi, Chengdu Shi',
  'Country': 'China'},
 {'City Name': 'shache',
  'Latitude': 38.4167,
  'Longitude': 77.2406,
  'Hotel Name': 'Delong Hotel',
  'Address': '2 齐乃,

# Done!